In [1]:
import pandas as pd
import numpy as np

base_path = "b2b_travel_userfraud_synth_v6_balanced_sticky"  # change if needed

# -----------------------------
# Load tables
# -----------------------------
users  = pd.read_csv(f"{base_path}/user_master.csv")
book   = pd.read_csv(f"{base_path}/booking_fact.csv")
label  = pd.read_csv(f"{base_path}/booking_label_table.csv")
events = pd.read_csv(f"{base_path}/post_booking_events.csv")
ipm    = pd.read_csv(f"{base_path}/ip_master.csv")

# -----------------------------
# Merge booking-level view
# -----------------------------
df = (book.merge(label, on="booking_id", how="left")
          .merge(events, on="booking_id", how="left")
          .merge(users[["user_id","user_fraud_label","user_fraud_type",
                       "user_age_days","avg_logins_per_day","failed_login_ratio"]],
                 on="user_id", how="left")
          .merge(ipm[["ip_id","is_proxy"]], on="ip_id", how="left"))

df["proxy_flag"] = df["is_proxy"].fillna(0).astype(int)

# -----------------------------
# User-level metrics table
# -----------------------------
user_metrics = (
    df.groupby("user_id")
      .agg(
          user_fraud_label=("user_fraud_label","max"),
          user_fraud_type=("user_fraud_type","max"),
          user_age_days=("user_age_days","max"),
          avg_logins=("avg_logins_per_day","max"),
          failed_login_ratio=("failed_login_ratio","max"),

          total_bookings=("booking_id","count"),
          fraud_bookings=("fraud_label","sum"),

          cancels=("is_cancelled","sum"),
          disputes=("is_disputed","sum"),

          total_loss=("final_loss_amount","sum"),
          avg_booking_value=("booking_value","mean"),
          max_booking_value=("booking_value","max"),

          proxy_rate=("proxy_flag","mean"),
          unique_ips=("ip_id","nunique"),
      )
      .reset_index()
)

user_metrics["cancel_rate"] = (user_metrics["cancels"] / user_metrics["total_bookings"]).round(3)
user_metrics["dispute_rate"] = (user_metrics["disputes"] / user_metrics["total_bookings"]).round(3)
user_metrics["proxy_rate_pct"] = (user_metrics["proxy_rate"] * 100).round(2)

fraud_users = user_metrics[user_metrics["user_fraud_label"] == 1].copy()
good_users  = user_metrics[user_metrics["user_fraud_label"] == 0].copy()

print("✅ Fraud users:", len(fraud_users))
print("✅ Good users :", len(good_users))

# -----------------------------
# Relevant columns per fraud type (ONLY)
# -----------------------------
cols_map = {
    "cancellation_abuser": [
        "user_id", "total_bookings", "cancels", "cancel_rate"
    ],
    "account_takeover": [
        "user_id", "failed_login_ratio", "avg_logins", "unique_ips",
        "proxy_rate_pct", "fraud_bookings"
    ],
    "new_synthetic_user": [
        "user_id", "user_age_days", "total_bookings", "fraud_bookings",
        "proxy_rate_pct", "unique_ips"
    ],
    "credit_bustout_user": [
        "user_id", "fraud_bookings", "max_booking_value",
        "avg_booking_value", "disputes", "total_loss"
    ],
    "bot_booking": [
        "user_id", "avg_logins", "total_bookings", "fraud_bookings", "unique_ips"
    ],
    "ring_operator": [
        "user_id", "fraud_bookings", "unique_ips", "proxy_rate_pct", "total_bookings"
    ],
}

# -----------------------------
# Print FRAUD users (top examples) + GOOD baseline average for same columns
# -----------------------------
def print_user_type_compare(ftype, top_n=8):
    fraud_sub = fraud_users[fraud_users["user_fraud_type"] == ftype].copy()
    if fraud_sub.empty:
        print(f"\n❌ No fraud users found for: {ftype}")
        return

    # Sort per fraud type (so eye-test makes sense)
    if ftype == "cancellation_abuser":
        fraud_sub = fraud_sub.sort_values(["cancel_rate", "cancels"], ascending=False)
    elif ftype == "credit_bustout_user":
        fraud_sub = fraud_sub.sort_values(["total_loss", "max_booking_value"], ascending=False)
    elif ftype == "account_takeover":
        fraud_sub = fraud_sub.sort_values(["failed_login_ratio", "unique_ips"], ascending=False)
    elif ftype == "new_synthetic_user":
        fraud_sub = fraud_sub.sort_values(["user_age_days", "total_bookings"], ascending=[True, False])
    elif ftype == "bot_booking":
        fraud_sub = fraud_sub.sort_values(["avg_logins", "total_bookings"], ascending=False)
    elif ftype == "ring_operator":
        fraud_sub = fraud_sub.sort_values(["unique_ips", "proxy_rate_pct"], ascending=False)

    cols = cols_map[ftype]

    print("\n" + "="*110)
    print(f"USER FRAUD TYPE: {ftype.upper()}")
    print("="*110)

    print("\n[FRAUD USERS - SAMPLE]")
    print(fraud_sub[cols].head(top_n).to_string(index=False))

    # GOOD baseline = average of those same metrics (excluding user_id)
    metric_cols = [c for c in cols if c != "user_id"]
    good_avg = good_users[metric_cols].mean(numeric_only=True).to_frame().T
    good_avg.insert(0, "baseline_group", "GOOD_USERS_AVG")

    # format small decimals
    for c in good_avg.columns:
        if c in ["failed_login_ratio", "cancel_rate", "dispute_rate"]:
            good_avg[c] = good_avg[c].round(4)
        elif c.endswith("_pct"):
            good_avg[c] = good_avg[c].round(2)
        else:
            good_avg[c] = good_avg[c].round(3)

    print("\n[GOOD USERS BASELINE - AVERAGE]")
    print(good_avg.to_string(index=False))

for ftype in cols_map.keys():
    print_user_type_compare(ftype, top_n=8)

✅ Fraud users: 32
✅ Good users : 760

USER FRAUD TYPE: CANCELLATION_ABUSER

[FRAUD USERS - SAMPLE]
user_id  total_bookings  cancels  cancel_rate
 U00301              14       14        1.000
 U00538              31       26        0.839
 U00275              17       14        0.824
 U00339              22       18        0.818

[GOOD USERS BASELINE - AVERAGE]
baseline_group  total_bookings  cancels  cancel_rate
GOOD_USERS_AVG           9.579    0.547        0.057

USER FRAUD TYPE: ACCOUNT_TAKEOVER

[FRAUD USERS - SAMPLE]
user_id  failed_login_ratio  avg_logins  unique_ips  proxy_rate_pct  fraud_bookings
 U00780              0.5299       16.64          13           47.62              21
 U00185              0.5265       15.67          10           66.67              21
 U00737              0.4898       11.40          10           58.33              12
 U00366              0.4735       15.40          12           63.16              19
 U00696              0.4467       20.93          13  